# Multiprocessing
Tutorial by Jonas Wilfert, Tobias Thummerer

## License
Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons, Josef Kircher, Johannes Stoljar, Jonas Wilfert

Licensed under the MIT license. See [LICENSE](https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

## Motivation
This Julia Package *FMI.jl* is motivated by the use of simulation models in Julia. Here the FMI specification is implemented. FMI (*Functional Mock-up Interface*) is a free standard ([fmi-standard.org](http://fmi-standard.org/)) that defines a container and an interface to exchange dynamic models using a combination of XML files, binaries and C code zipped into a single file. The user can thus use simulation models in the form of an FMU (*Functional Mock-up Units*). Besides loading the FMU, the user can also set values for parameters and states and simulate the FMU both as co-simulation and model exchange simulation.

## Introduction to the example
This example shows how to parallelize the computation of an FMU in FMI.jl. We can compute a batch of FMU-evaluations in parallel with different initial settings.
Parallelization can be achieved using multithreading or using multiprocessing. This example shows **multiprocessing**, check `multithreading.ipynb` for multithreading.
Advantage of multithreading is a lower communication overhead as well as lower RAM usage.
However in some cases multiprocessing can be faster as the garbage collector is not shared.


The model used is a one-dimensional spring pendulum with friction. The object-orientated structure of the *SpringFrictionPendulum1D* can be seen in the following graphic.

![svg](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/pics/SpringFrictionPendulum1D.svg?raw=true)  


## Target group
The example is primarily intended for users who work in the field of simulations. The example wants to show how simple it is to use FMUs in Julia.


## Other formats
Besides, this [Jupyter Notebook](https://github.com/thummeto/FMI.jl/blob/examples/examples/multiprocessing.ipynb) there is also a [Julia file](https://github.com/thummeto/FMI.jl/blob/examples/examples/multiprocessing.jl) with the same name, which contains only the code cells and for the documentation there is a [Markdown file](https://github.com/thummeto/FMI.jl/blob/examples/examples/multiprocessing.md) corresponding to the notebook.  


## Getting started

### Installation prerequisites
|     | Description                       | Command                   | Alternative                                    |   
|:----|:----------------------------------|:--------------------------|:-----------------------------------------------|
| 1.  | Enter Package Manager via         | ]                         |                                                |
| 2.  | Install FMI via                   | add FMI                   | add " https://github.com/ThummeTo/FMI.jl "     |
| 3.  | Install FMIZoo via                | add FMIZoo                | add " https://github.com/ThummeTo/FMIZoo.jl "  |
| 4.  | Install FMICore via               | add FMICore               | add " https://github.com/ThummeTo/FMICore.jl " |
| 5.  | Install BenchmarkTools via        | add BenchmarkTools        |                                                |

## Code section



Adding your desired amount of processes:

In [1]:
using Distributed
n_procs = 4
addprocs(n_procs; exeflags=`--project=$(Base.active_project()) --threads=auto`, restrict=false)

4-element Vector{Int64}:
 2
 3
 4
 5

To run the example, the previously installed packages must be included. 

In [2]:
# imports
@everywhere using FMI
@everywhere using FMIZoo
@everywhere using BenchmarkTools

Checking that we workers have been correctly initialized:

In [3]:
workers()

@everywhere println("Hello World!")

# The following lines can be uncommented for more advanced informations about the subprocesses
# @everywhere println(pwd())
# @everywhere println(Base.active_project())
# @everywhere println(gethostname())
# @everywhere println(VERSION)
# @everywhere println(Threads.nthreads())

      From worker 2:	Hello World!
Hello World!
      From worker 4:	Hello World!
      From worker 3:	Hello World!
      From worker 5:	Hello World!


### Simulation setup

Next, the batch size and input values are defined.

In [4]:

# Best if batchSize is a multiple of the threads/cores
batchSize = 16

# Define an array of arrays randomly
input_values = collect(collect.(eachrow(rand(batchSize,2))))

16-element Vector{Vector{Float64}}:
 [0.983811111868945, 0.8047252036376396]
 [0.0631047007458827, 0.13161014652578196]
 [0.9771908967464737, 0.6184758169549578]
 [0.8565687358631271, 0.7263367911027785]
 [0.05219904798384034, 0.02453186443854527]
 [0.1965121161864294, 0.9397560888639491]
 [0.6712264905165723, 0.8107279248346988]
 [0.024618663843076538, 0.7417181164749418]
 [0.18900966693614696, 0.8290925433277117]
 [0.49058360065629625, 0.9924508065199722]
 [0.7633121745682394, 0.0944304588150029]
 [0.13432471935597823, 0.03457915931257216]
 [0.49654604984910833, 0.4000890227124094]
 [0.8474239111616684, 0.19233282731327384]
 [0.6782617314045774, 0.7224374471188286]
 [0.045868340973776345, 0.6937776228945121]

### Shared Module
For Distributed we need to embed the FMU into its own `module`. This prevents Distributed from trying to serialize and send the FMU over the network, as this can cause issues. This module needs to be made available on all processes using `@everywhere`.

In [5]:
@everywhere module SharedModule
    using FMIZoo
    using FMI

    t_start = 0.0
    t_step = 0.1
    t_stop = 10.0
    tspan = (t_start, t_stop)
    tData = collect(t_start:t_step:t_stop)

    model_fmu = FMIZoo.fmiLoad("SpringPendulum1D", "Dymola", "2022x")
end

┌ Info: fmi2Unzip(...): Successfully unzipped 153 files at `/tmp/fmijl_1HKlfy/SpringPendulum1D`.
└ @ FMIImport /home/runner/.julia/packages/FMIImport/4SJhD/src/FMI2_ext.jl:76
[ Info: fmi2Unzip(...): Successfully unzipped 153 files at `/tmp/fmijl_1PBM8W/SpringPendulum1D`.
┌ Info: fmi2Load(...): FMU resources location is `file:////tmp/fmijl_1HKlfy/SpringPendulum1D/resources`
└ @ FMIImport /home/runner/.julia/packages/FMIImport/4SJhD/src/FMI2_ext.jl:192
┌ Info: fmi2Load(...): FMU supports both CS and ME, using CS as default if nothing specified.
└ @ FMIImport /home/runner/.julia/packages/FMIImport/4SJhD/src/FMI2_ext.jl:195
[ Info: fmi2Load(...): FMU resources location is `file:////tmp/fmijl_1PBM8W/SpringPendulum1D/resources`
[ Info: fmi2Load(...): FMU supports both CS and ME, using CS as default if nothing specified.
[ Info: fmi2Unzip(...): Successfully unzipped 153 files at `/tmp/fmijl_MgK31N/SpringPendulum1D`.
[ Info: fmi2Unzip(...): Successfully unzipped 153 files at `/tmp/fmijl_AC4mhT

We define a helper function to calculate the FMU and combine it into an Matrix.

In [6]:
@everywhere function runCalcFormatted(fmu, x0, recordValues=["mass.s", "mass.v"])
    data = fmiSimulateME(fmu, SharedModule.t_start, SharedModule.t_stop; recordValues=recordValues, saveat=SharedModule.tData, x0=x0, showProgress=false, dtmax=1e-4)
    return reduce(hcat, data.states.u)
end

Running a single evaluation is pretty quick, therefore the speed can be better tested with BenchmarkTools.

In [7]:
@benchmark data = runCalcFormatted(SharedModule.model_fmu, rand(2))

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  692.359 ms … 717.231 ms  ┊ GC (min … max): 3.39% … 3.02%
 Time  (median):     701.408 ms               ┊ GC (median):    3.30%
 Time  (mean ± σ):   702.643 ms ±   7.420 ms  ┊ GC (mean ± σ):  3.73% ± 0.81%

  ▁         ▁         █   ▁   ▁       ▁                       ▁  
  █▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  692 ms           Histogram: frequency by time          717 ms <

 Memory estimate: 155.95 MiB, allocs estimate: 3602373.

### Single Threaded Batch Execution
To compute a batch we can collect multiple evaluations. In a single threaded context we can use the same FMU for every call.

In [8]:
println("Single Threaded")
@benchmark collect(runCalcFormatted(SharedModule.model_fmu, i) for i in input_values)

Single Threaded


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 11.297 s (4.12% GC) to evaluate,
 with a memory estimate of 2.44 GiB, over 57637956 allocations.

### Multithreaded Batch Execution
In a multithreaded context we have to provide each thread it's own fmu, as they are not thread safe.
To spread the execution of a function to multiple processes, the function `pmap` can be used.

In [9]:
println("Multi Threaded")
@benchmark pmap(i -> runCalcFormatted(SharedModule.model_fmu, i), input_values)

Multi Threaded


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 6.947 s (0.00% GC) to evaluate,
 with a memory estimate of 84.70 KiB, over 1311 allocations.

As you can see, there is a significant speed-up in the median execution time. But: The speed-up is often much smaller than `n_procs` (or the number of physical cores of your CPU), this has different reasons. For a rule of thumb, the speed-up should be around `n/2` on a `n`-core-processor with `n` Julia processes.

### Unload FMU

After calculating the data, the FMU is unloaded and all unpacked data on disc is removed.

In [10]:
@everywhere fmiUnload(SharedModule.model_fmu)

### Summary

In this tutorial it is shown how multi processing with `Distributed.jl` can be used to improve the performance for calculating a Batch of FMUs.